# 유사 감사 사례 검색 & 처분 수준 추천

## 검색 로직 설계
- Embedding
- Retriever
- Rerank
- Similarity Calculation
- Retrieval Performance

In [6]:
# 환경변수 접근
import os
from dotenv import load_dotenv

from langchain.document_loaders import DirectoryLoader # 디렉토리 불러오기
from langchain_community.document_loaders import PyPDFLoader # PDF 파일 불러오기
from langchain.text_splitter import RecursiveCharacterTextSplitter # 텍스트 분할

# 임베딩, LLM
import time # API 호출 속도 조절을 위해 sleep 사용
from langchain_naver import ClovaXEmbeddings, ChatClovaX 
from langchain_openai import OpenAIEmbeddings, ChatOpenAI 

# VectorDB
from langchain_chroma import Chroma
from chromadb import PersistentClient
from qdrant_client import QdrantClient
from qdrant_client.http import models
from langchain_qdrant import Qdrant

# Langchain Core
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# 성능 평가
import numpy as np # 평균 계산
import pandas as pd # DataFrame 형식으로 변환
from datasets import Dataset # Ragas 평가용 Dataset 객체로 변환
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision # Ground Truth 없이 평가 가능한 2가지 핵심 지표 Faithfulness(환각 감소), Precision(노이즈 감소)측정 

In [7]:
load_dotenv() # 환경변수 불러오기

True

## [VectorDB]
- Qdrant가 ChromaDB에 비해 안정적이며 필터링 직관적
- elastic search

## API 사용
- ChatOpenAI
    - 호환성 높음
    - Reranker 없음
- ChatClovaX
    - Reranker 제공
    - 한국어 모델
    - 최신 Langchain 호환 불가

In [8]:
# # 감사원에서 받아온 파일 모두 불러오기 (10분 소요)
# directory_path = '../data/1_bai_raw_files/'

# loader = DirectoryLoader(
#     directory_path,
#     glob='*.pdf',
#     loader_cls=PyPDFLoader,
# )

# documents = loader.load()

In [9]:
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=512,
#     chunk_overlap=128,
#     separators=['\n\n', '\n', '.', ' ', '']
# )

In [10]:
# docs = text_splitter.split_documents(documents)

### 로컬 저장소에 DB 구축

##### batch 나눠서 진행하는 이유?
- HyperClova
    - RateLimitError: docs객체를 한 번에 임베딩하면 API가 처리할 수 있는 분당 최대 토큰 수 초과
- OpenAI
    - BadRequestError: 한 번의 API요청에 담긴 전체 토큰 수가 API 서버의 최대 토큰 제한 초과

### [임베딩 모델]
- HyperClova
    - **BGE-M3: 가장 최신의 다국어 임베딩 모델 (선택)**
    - C-SBERT-V2: CLOVA 자체 최적화한 BERT 기반 모델
- OpenAI
    - text-embedding-3-large: 고성능 임베딩 모델이나, small에 비해 약 4배 비쌈
    - **text-embedding-3-small (선택)**

In [11]:
# # 임베딩 후 DB 저장하는 함수
# def custom_embed_with_sleep(docs, embedding_model, persist_path, collection_name, batch_size=100, sleep_time=0.1):
#     vectorstore = None

#     client = PersistentClient(path=persist_path)
#     total_batches = len(docs) // batch_size + (1 if len(docs) % batch_size != 0 else 0)
    
#     for i in range(0, len(docs), batch_size):
#         batch_docs = docs[i:i + batch_size]
        
#         # 첫 번째 배치에서만 DB 생성
#         if i == 0:
#             print(f"Creating new Chroma DB at: {persist_path} with the first batch...")
#             vectorstore = Chroma.from_documents(
#                 batch_docs, 
#                 embedding_model, 
#                 client=client,
#                 collection_name=collection_name
#             )
#         else:
#             # 이후 배치들은 기존 DB 인스턴스에 추가
#             vectorstore.add_documents(batch_docs)
        
#         current_batch_num = i // batch_size + 1
#         print(f"Processed batch {current_batch_num}/{total_batches}. Sleeping for {sleep_time} seconds...")
#         time.sleep(sleep_time) # 지연 시간 추가
        
#     return vectorstore

In [12]:
# # 감사원 데이터 HyperClova 임베딩 모델 사용해 DB 저장
# hc_vectorstore = custom_embed_with_sleep(
#     docs, # 저장할 문서
#     ClovaXEmbeddings(model='bge-m3'), # 사용할 임베딩 모델
#     '../data/bai_hc',     # 저장 경로
#     'hc_audit_collection'    # collection name
# )

In [13]:
# # 감사원 데이터 OpenAI 임베딩 모델 사용해 DB 저장
# openai_vectorstore = custom_embed_with_sleep(
#     docs, # 저장할 문서
#     OpenAIEmbeddings(model='text-embedding-3-small'), # 사용할 임베딩 모델
#     '../data/bai_openai',     # 저장 경로 
#     'openai_audit_collection'    # collection name
# )

##### VectorDB 구축 소요 시간
1.  HyperClova: 약 1시간
2. OpenAI: 약 10분

- 원인
    - API 호출 속도
    - 배치 처리 효율성

In [14]:
# 감사원 DB 불러오기
hc_db = Chroma(persist_directory='../data/bai_hc', collection_name='hc_audit_collection', embedding_function=ClovaXEmbeddings(model='bge-m3'))
openai_db = Chroma(persist_directory='../data/bai_openai', collection_name='openai_audit_collection', embedding_function=OpenAIEmbeddings(model='text-embedding-3-small'))

### [Retriever]
- similarity: 유클리드 거리가 가깝거나, 코사인 유사도가 높은 상위 k개의 문서 반환 (유클리드, 코사인 선택은 DB 생성 시 결정)
- mmr: 유사성 뿐만 아니라 다양성 고려하여 문서 선택
- similarity_score_threshold: 특정 임계값 이상인 문서 중 k개 문서 반환

- similarity => 가장 흔한, 일반적인 처분 사례만 찾아와 판단 스펙트럼이 좁아짐
- mmr => 일반적인 처분 사례와 더불어, 흔치는 않지만 나올 수 있는 처분의 최소기준과 최대 기준 파악 가능

- SelfQuery Retriever
    - 장점: 필터에 맞는 문서들 내에서만 유사성 검색 수행하여, 메타데이터 필터링이 필수적이거나 복잡한 조건이 필요한 경우 유용
    - 단점: 사용자 질문에서 필터 조건과 쿼리 추출하기 위해 LLM 한 번 더 호출해야 함. 사용자 질문 잘못 해석하거나 존재하지 않는 메타데이터 키를 사용하여 관련된 문서 제외시킬 수 있음.
    - ex) 2024년 1분기에 발생한 법인카드 유용 사례 찾아줘
        - LLM은 metadata['date'] > '2024-01-01' AND metadata['category'] =='법인카드' 생성
        - 쿼리는 "유용 사례" 만 vectorstore에 전달
        - 필터에 맞는 문서들 내에서만 유사성 검색 수행하여, 메타데이터 필터링이 필수적이거나 복잡한 조건이 필요한 경우 사용
- MultiQuery Retriever
    - 장점: 단일 질문에 대해 여러 관점을 확보하여 검색 실패 위험 줄임.
    - 단점: 사용자의 질문을 여러 개의 새로운 쿼리로 변환하기 위해 LLM 한 번 더 호출해야 함. 의도와 동떨어진 노이즈 쿼리 생성 가능.
- MultiVector Retriever
    - 검색 단계에서 작은 벡터 사용하고, 생성 단계에서 원본의 큰 청크를 다시 불러오는 추가 조회.
    - 단점: 생성단계에서 원본의 큰 청크 다시 불러오는 추가 조회 필요.

- 각각 비용, 시간 증가 (2배가 됨)

##### 선택
- search type
    - mmr
- Retriever
    - 메타데이터 필터링 구체화 가능하다? => SelfQueryRetriever, MultiVectorRetriever
    - 메타데이터 필터링 자체가 성능 악화 가능성 높아보인다? => MultiVectorRetriever

In [16]:
import langchain_naver

In [17]:
dir(langchain_naver)

['ChatClovaX',
 'ClovaXEmbeddings',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'chat_models',
 'const',
 'embeddings']

In [38]:
# HyperClova Retriever 설정
hc_retriever = hc_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 5, # 최종적으로 반환할 문서의 수
        'fetch_k': 20, # 후보로 가져올 문서의 수
        'lambda_mult': 0.6 # 유사도, 다양성 비율(0: 다양하게, 1: 유사한것만)
    }
)

In [39]:
# OpenAI Retriever 설정
openai_retriever = openai_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 5, # 최종적으로 반환할 문서의 수
        'fetch_k': 20, # 후보로 가져올 문서의 수
        'lambda_mult': 0.6 # 유사도, 다양성 비율(0: 다양하게, 1: 유사한것만)
    }
)

### [LLM 모델]

- HyperClova
    - HCX DASH 계열: 경량화 모델
    - **HCX 계열: 고성능 모델 (선택)**
- OpenAI
    - gpt-3.5-turbo
    - **gpt-4o-mini (선택)**

In [40]:
hc_llm = ChatClovaX(model='HCX-007', temperature=0) # HyperClova LLM

In [41]:
openai_llm = ChatOpenAI(model='gpt-4o-mini', temperature=0) # OpenAI LLM

### [Prompt 설정]

In [116]:
recommend_prompt = """
당신은 감사 조직의 처분 수준 추천 전문 에이전트입니다.
당신의 임무는 사용자로부터 입력받은 **감사 사례**에 대해, 아래 **[검색된 유사 사례와 처분 기록]**을 엄격하게 분석하고, 아래 **[감사 결과 처리 기준 법률 (제25조)]**에 의거하여 가장 합리적이고 설명 가능한 **처분 수준**을 추천하는 것입니다.

### 🌟 감사 결과 처리 기준 법률 (제25조)
처분 수준은 다음 각 호의 기준 중 하나를 선택해야 합니다.

1.  **변상명령**: 「회계관계직원 등의 책임에 관한 법률」이 정하는 바에 따라 변상책임이 있는 경우.
2.  **징계 또는 문책요구**: 「국가공무원법」과 그 밖의 법령에 규정된 징계 또는 문책 사유에 해당하는 경우. (가장 강함)
3.  **고발**: 감사 결과 범죄 혐의가 있다고 인정되는 경우.
4.  **시정요구**: 감사 결과 위법 또는 부당하다고 인정되는 사실이 있어 추징ㆍ회수ㆍ환급ㆍ추급 또는 원상복구 등이 필요한 경우.
5.  **개선요구**: 감사 결과 법령상ㆍ제도상 또는 행정상 모순이 있거나 개선할 사항이 있다고 인정되는 경우.
6.  **권고**: 감사 결과 문제점이 인정되는 사실이 있어 그 대안을 제시하고 개선방안을 마련하도록 할 필요가 있는 경우.
7.  **통보**: 감사 결과 비위 사실이나 위법 또는 부당하다고 인정되는 사실이 있으나 다른 요구를 하기에 부적합하여 각 기관 또는 부서에서 자율적으로 처리할 필요가 있는 경우.
8.  **주의요구**: 위법성 또는 부당성의 경중에 따라 다음으로 세분화하여 선택합니다.
    * **경고**: 위법성 또는 부당성의 정도가 징계 사유에 이르지 않으나, 공식적인 기록에 남는 제재가 필요한 경우.
    * **훈계**: 위법성 또는 부당성의 정도가 경미하여 제재보다는 재발 방지 교육 등으로 충분한 경우. (가장 약함)

### 핵심 지침

1.  **법률 최우선:** 추천 처분 수준은 반드시 위에 명시된 **제25조의 각 호** 중 하나(징계/고발/변상/시정/주의(경고/훈계)/통보/개선/권고)를 선택해야 합니다.
2.  **유사 사례 인용:** 모든 분석과 추천은 반드시 **[검색된 유사 사례와 처분 기록]({context})**에 있는 구체적인 정보를 인용하여 법률적 판단의 근거로 뒷받침해야 합니다.
3.  **위반 내역 및 처분 요약:** `{context}`를 기반으로 과거 유사 위반 내역과 해당 처분 결과를 간결하게 요약합니다.

### 응답 형식

모든 응답은 아래의 **마크다운 구조**를 엄격하게 준수해야 합니다.

## ⚖️ [새로운 감사 사례] 분석 및 처분 수준 추천

### 1. 유사 사례 분석 및 과거 처분 요약
[검색된 유사 사례들({context})을 분석하여 핵심 위반 내용, 규모, 해당 처분 결과를 3줄 이내로 요약]

### 2. 추천 처분 수준 및 근거
**[추천 처분 수준]**: [선택한 처분 수준 (예: 징계 또는 문책요구, 주의요구(경고), 통보)]

**[법률적 근거]**
[선택한 처분 수준이 제25조의 어느 호에 해당하는지, 해당 호의 법률적 요건을 인용하여 설명]

**[추론 및 상세 근거]**
1. **유사성 인용:** [가장 유사한 과거 사례와 그 처분 수준을 구체적으로 인용]
2. **심각성 비교:** [새로운 사례가 인용된 과거 사례와 비교하여 제25조의 요건에 어떻게 부합하는지, 그리고 왜 이 처분 수준이 적합한지 논리적으로 설명]
3. **최종 판단:** [법률적 기준과 유사 사례 비교를 통한 최종 처분 추천 이유 요약]
"""

In [117]:
# recommend_prompt = """
# 당신은 감사 조직의 처분 수준 추천 전문 에이전트입니다.
# 당신의 임무는 사용자로부터 입력받은 **감사 사례**에 대해, 아래 **[검색된 유사 사례와 처분 기록]**을 엄격하게 분석하여 가장 합리적이고 설명 가능한 **처분 수준**을 추천하는 것입니다.

# ### 핵심 지침

# 1.  **유사 사례 인용:** 모든 분석과 추천은 반드시 **[검색된 유사 사례와 처분 기록]({context})**에 있는 구체적인 정보를 인용하여 법률적 판단의 근거로 뒷받침해야 합니다.
# 2.  **위반 내역 및 처분 요약:** `{context}`를 기반으로 과거 유사 위반 내역과 해당 처분 결과를 간결하게 요약합니다.

# ### 응답 형식

# 모든 응답은 아래의 **마크다운 구조**를 엄격하게 준수해야 합니다.

# ## ⚖️ [새로운 감사 사례] 분석 및 처분 수준 추천

# ### 1. 유사 사례 분석 및 과거 처분 요약
# [검색된 유사 사례들({context})을 분석하여 핵심 위반 내용, 규모, 해당 처분 결과를 3줄 이내로 요약]

# ### 2. 추천 처분 수준 및 근거
# **[추천 처분 수준]**: [선택한 처분 수준 (예: 징계 또는 문책요구, 주의요구(경고), 통보)]

# **[추론 및 상세 근거]**
# 1. **유사성 인용:** [가장 유사한 과거 사례와 그 처분 수준을 구체적으로 인용]
# 2. **심각성 비교:** [새로운 사례가 인용된 과거 사례와 비교하여 왜 이 처분 수준이 적합한지 논리적으로 설명]
# 3. **최종 판단:** [법률적 기준과 유사 사례 비교를 통한 최종 처분 추천 이유 요약]
# """

In [118]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', recommend_prompt),
        ('human', '제시한 감사 사례: {question} \n\n [검색된 유사 사례와 처분 기록]:\n{context}')
    ]
)

### [Chain 구성]

In [119]:
# 검색된 문서 객체 리스트를 하나의 문자열로 결합하여 LLM이 쉽게 이해할 수 있도록 전달
def format_docs(docs):
    return "\n\n---\n\n".join(doc.page_content for doc in docs)

In [120]:
from operator import itemgetter

hc_chain = (
    RunnableLambda(lambda x: {'question': x})
    |
    {"context": itemgetter("question") | hc_retriever | RunnableLambda(format_docs),
    "question": itemgetter("question")}
    | qa_prompt
    | hc_llm
    | StrOutputParser()
)

openai_chain = (
    RunnableLambda(lambda x: {'question': x})
    |
    {"context": itemgetter("question") | openai_retriever | RunnableLambda(format_docs),
    "question": itemgetter("question")}
    | qa_prompt
    | openai_llm
    | StrOutputParser()
)

### [Test]

[변화]
- hc_chain, openai_chain의 성능을 확인
- 시스템 프롬프트 수정
- chain 구조 수정

In [68]:
# 프롬프트에 처분 기준 조항 미포함 시
answer = hc_chain.invoke('직무관련자로부터 금품 수수를 받은 경우 어떤 처분을 내려야해?')
print(answer)

## ⚖️ 직무관련자로부터 금품 수수를 받은 경우 처분 수준 추천  

### 1. 유사 사례 분석 및 과거 처분 요약  
- **제2조 규정**: 금품 수수액이 **200만 원 이상**인 경우 **사법기관 고발 필수**, 내부징계와 병행.  
- **LH 사례(A)**: 총 **154백만 원**(약 1.54억 원) 수수 → 징계처분 권고됨.  
- **특허청(BF)**: 직무관련 업체 금품 수수 → 징계처분 + 재발 시 성과급 제한/사업참여 배제 검토.  

### 2. 추천 처분 수준 및 근거  
**[추천 처분 수준]**: **통보**  

**[추론 및 상세 근거]**  
1. **규정 직접 적용**: 금품 수수액(**154백만 원**)이 **200만 원 이상**이므로 **고발 의무화** 대상.  
2. **유사 사례 대비 심각성**: LH 사례(A)와 동일하게 대규모 금품 수수 → 징계+고발이 적절.  
3. **추가 조치 필요성**: 재발을 막기 위해 성과급 제한, 계약 참여 배제 등 제재 강화 권장.  

→ **통보** 처분은 법적 고발 요건 충족뿐 아니라 조직적 예방 차원에서 최우선되어야 함.


In [76]:
# 프롬프트에 처분 기준 조항 포함 시
answer = hc_chain.invoke('직무관련자로부터 금품 수수를 받은 경우 어떤 처분을 내려야해?')
print(answer)

### 1. 유사 사례 분석 및 과거 처분 요약  
유사 사례(LH 직원 A의 154백만 원 금품 수수)에서 **징계처분**과 함께 **사법기관 고발**이 이루어졌으며, 재발 방지를 위한 성과급 제한·사업참여 배제 등 추가 조치가 검토됨. 또한, 금품 수수액이 **200만 원 이상**인 경우 **필수 고발 대상**임을 명시함.  

---

### 2. 추천 처분 수준 및 근거  
**[추천 처분 수준]:** **징계 또는 문책요구** + **고발**  

**[법률적 근거]**  
- 제2조 제2항: 금품 수수 시 **내부징계**와 **병행 고발** 의무 (단, 수수액 ≥ 200만 원 시 **필수 고발**).  
- 대법원 판례: 직무관련성 범위 확대 해석 (법령상 권한 포함).  

**[추론 및 상세 근거]**  
1. **유사성 인용**: LH 사례처럼 대규모 금품 수수(154백만 원) 시 **징계+고발** 조합 적용.  
2. **심각성 비교**: 현재 사례의 구체적 금액 불명확하나, 유사 패턴(직무관련자와 공모, 반복적 수수)으로 **징계 및 고발 요건 충족** 추정.  
3. **최종 판단**: 금품 수수 고의성·규모 감안 시 **징계**로 책임 묻고, 법적 처벌 필요성으로 **고발** 병행이 합리적.  

--- 

✂️ **추가 권고사항**:  
- 금액 확인 후 200만 원 미만 시 **징계** 단독 적용 가능.  
- 재발을 막기 위해 **성과급 제한·사업참여 배제** 등 추가 조치 도입 권장.


In [69]:
# 프롬프트에 처분 기준 조항 미포함 시
answer = openai_chain.invoke('직무관련자로부터 금품 수수를 받은 경우 어떤 처분을 내려야해?')
print(answer)

## ⚖️ [새로운 감사 사례] 분석 및 처분 수준 추천

### 1. 유사 사례 분석 및 과거 처분 요약
직무 관련자로부터 금품을 수수한 경우는 다음과 같은 처분 기준이 있습니다. 
- **금액 100만 원 미만**: 동감봉, 정직, 강등
- **금액 100만 원 이상 300만 원 미만**: 강등, 해임
- **금액 300만 원 이상 500만 원 미만**: 해임
- **금액 500만 원 이상**: 파면

이와 같은 기준에 따라, 직무와 관련하여 금품을 수수한 경우, 위법하거나 부당한 처분을 하지 않은 경우에도 동정직, 강등, 해임 등의 처분이 내려질 수 있습니다. 

### 2. 추천 처분 수준 및 근거
**[추천 처분 수준]**: 통보

**[추론 및 상세 근거]**
1. **유사성 인용:** 직무 관련자로부터 금품을 수수한 경우는 과거 사례에서 금액에 따라 강등, 해임, 파면 등의 처분이 내려졌습니다. 특히, 위법한 처분이 없었던 경우에도 동정직이나 강등이 이루어졌습니다.
2. **심각성 비교:** 새로운 사례가 금품 수수의 경우로, 직무와 관련된 금품 수수는 일반적으로 심각한 위반으로 간주됩니다. 따라서, 과거 사례와 비교했을 때 심각성이 높다고 판단됩니다.
3. **최종 판단:** 직무 관련자로부터 금품을 수수한 경우는 중대한 위반으로, 통보 처분이 적절하다고 판단됩니다. 이는 향후 재발 방지를 위한 조치로, 수사기관에 통보하여 추가적인 조치를 취할 수 있도록 하는 것이 필요합니다.


In [77]:
# 프롬프트에 처분 기준 조항 포함 시
answer = openai_chain.invoke('직무관련자로부터 금품 수수를 받은 경우 어떤 처분을 내려야해?')
print(answer)

## ⚖️ [새로운 감사 사례] 분석 및 처분 수준 추천

### 1. 유사 사례 분석 및 과거 처분 요약
직무 관련자로부터 금품을 수수한 경우는 다음과 같은 처분 기준이 있습니다. 
- **100만 원 미만**: 동감봉, 정직, 강등
- **100만 원 이상 300만 원 미만**: 강등, 해임
- **300만 원 이상 500만 원 미만**: 해임, 파면
- **500만 원 이상**: 파면

이와 같은 기준에 따라, 직무와 관련하여 금품을 수수한 경우는 위법한 처분을 하지 않은 경우에도 징계가 필요하며, 위법한 처분을 한 경우에는 더욱 강력한 처분이 요구됩니다.

### 2. 추천 처분 수준 및 근거
**[추천 처분 수준]**: 징계 또는 문책요구

**[법률적 근거]**
제25조 제2호에 따르면, "국가공무원법"과 그 밖의 법령에 규정된 징계 또는 문책 사유에 해당하는 경우에는 징계 또는 문책요구를 할 수 있습니다. 금품 수수는 명백한 비위 행위로 간주되며, 이는 징계 사유에 해당합니다.

**[추론 및 상세 근거]**
1. **유사성 인용:** 과거 사례에서 직무와 관련하여 금품을 수수한 경우, 100만 원 이상 300만 원 미만의 금품 수수에 대해 강등 또는 해임 처분이 내려진 사례가 있습니다.
2. **심각성 비교:** 새로운 사례에서 금품 수수의 규모가 100만 원 이상인 경우, 이는 과거 사례와 유사하며, 징계 또는 문책 요구가 적합합니다. 특히, 위법한 처분을 한 경우에는 더욱 강력한 처분이 필요합니다.
3. **최종 판단:** 법률적 기준과 유사 사례를 비교할 때, 직무 관련자로부터 금품을 수수한 행위는 명백한 비위 행위로 간주되며, 따라서 징계 또는 문책 요구가 적합하다고 판단됩니다.


In [70]:
# 프롬프트에 처분 기준 조항 미포함 시
answer = hc_chain.invoke('농업용 저수지 제방 안전등급 판정기준 불합리의 경우 어떤 처분을 내려야할까?')
print(answer)

## ⚖️ 농업용 저수지 제방 안전등급 판정기준 불합리 사례 분석 및 처분 수준 추천  

### 1. 유사 사례 분석 및 과거 처분 요약  
- **핵심 위반 내용**: 농어촌공사의 안전진단 지침 미준수 가능성. 필댐 제방의 홍수 월류 위험 시 ‘D등급’ 이하로 판정해야 하나 이를 누락하거나 부적절하게 평가한 정황.  
- **관련 법령**: 「농어촌정비법」 제18조, 저수지관리규정 제18조 및 제22조, 안전진단 지침 등에 의거한 안전등급 판정 절차 위반.  
- **과거 처분 추정**: 유사 사례에서 안전등급 오류를 시정하고 관계자 주의 환기 차원에서 **‘경고’** 또는 **‘통보’** 조치가 이루어졌을 것으로 예상됨.  

### 2. 추천 처분 수준 및 근거  
**[추천 처분 수준]**: **경고**  

**[추론 및 상세 근거]**  
1. **유사성 인용**: 안전등급 판정 오류는 저수지 붕괴 위험과 직결되며, 과거 유사 사례에서도 법령 미준수에 대해 **‘경고’** 이상의 처분이 적용되었을 가능성이 높음.  
2. **심각성 비교**: 현 사례는 잠재적 인명·재산 피해 유발 요인이 있으므로 단순 주의보다 높은 수위인 **‘경고’** 가 적절. 다만, 명백한 고의성이나 반복적 위반 증거가 없어 **‘통보’** 는 과도함.  
3. **최종 판단**: 안전등급 재점검 및 관계 기관 협의를 통한 시스템 개선을 유도하기 위해 **‘경고’** 를 권고하되, 향후 동일 오류 재발 시 강화된 조치를 검토해야 함.  

---  
✅ **처분 수준 선정 이유**: 공공 안전과 직결된 사안이지만, 초기 단계의 절차적 오류로 판단되어 **‘경고’** 로 시정 촉구.


In [78]:
# 프롬프트에 처분 기준 조항 포함 시
answer = hc_chain.invoke('농업용 저수지 제방 안전등급 판정기준 불합리의 경우 어떤 처분을 내려야할까?')
print(answer)

### 1. 유사 사례 분석 및 과거 처분 요약  
유사 사례에서 농어촌공사는 저수지 안전등급 판정 시 홍수 월류 가능성 확인 시 ‘D등급 이하’로 판정해야 함에도 불구하고, 이를 누락하거나 부적절하게 평가했을 가능성이 있습니다. 이로 인해 시설 안전 관리가 소홀해질 우려가 있으며, 관계 법령(「농어촌정비법」, 「저수지댐법」) 상 안전등급 판정 기준을 위반한 것으로 보입니다.  

### 2. 추천 처분 수준 및 근거  
**[추천 처분 수준]**: **시정요구**  

**[법률적 근거]**  
「농어촌정비법」 제18조 및 시행령 제26조에 따라 저수지 안전등급 판정은 객관적 기준에 따라 이루어져야 하며, 홍수 월류 가능성 검토 누락 등은 위법·부당한 사실로 간주됩니다.  

**[추론 및 상세 근거]**  
1. **유사성 인용**: 과거 사례에서 안전등급 판정 오류 시 즉시 시정 조치와 함께 재발 방지 대책이 요구되었습니다.  
2. **심각성 비교**: 현 사례는 시설물 안전에 직결되며, 단순 절차적 오류가 아닌 생명·재산 피해로 이어질 수 있으므로 시정요구가 필수적입니다.  
3. **최종 판단**: 안전등급 판정 과정의 오류로 인한 잠재적 위험을 제거하기 위해 즉각적인 시정 조치가 필요합니다.  

---  
**결론**: 안전등급 판정 기준 불합리로 인한 위험 요소를 해소하기 위해 **시정요구**가 적절합니다.


In [71]:
# 프롬프트에 처분 기준 조항 미포함 시
answer = openai_chain.invoke('농업용 저수지 제방 안전등급 판정기준 불합리의 경우 어떤 처분을 내려야할까?')
print(answer)

## ⚖️ [농업용 저수지 제방 안전등급 판정기준 불합리] 분석 및 처분 수준 추천

### 1. 유사 사례 분석 및 과거 처분 요약
농업용 저수지의 안전등급 판정기준이 불합리하다는 사례는 농어촌공사가 정밀안전진단을 수행하는 과정에서 발생한 문제로, 저수지의 개별 시설에 대한 상태 및 안전성 평가가 제대로 이루어지지 않았음을 나타냅니다. 이와 관련된 법령 및 지침에 따르면, 저수지의 안전등급은 A부터 E까지 판별되며, 이러한 기준이 제대로 적용되지 않을 경우 안전사고의 위험이 증가할 수 있습니다. 과거 유사 사례에서는 안전관리 기준 미비로 인해 경고 처분이 내려진 바 있습니다.

### 2. 추천 처분 수준 및 근거
**[추천 처분 수준]**: 경고

**[추론 및 상세 근거]**
1. **유사성 인용:** 과거 유사 사례에서 농업용 저수지의 안전관리 기준 미비로 인해 경고 처분이 내려진 사례가 있습니다. 이 경우, 안전등급 판정 기준이 불합리하다는 점에서 유사성을 지닙니다.
2. **심각성 비교:** 새로운 사례는 안전등급 판정 기준의 불합리함으로 인해 저수지의 안전성이 위협받을 수 있는 가능성을 내포하고 있으나, 직접적인 사고나 피해가 발생한 것은 아닙니다. 따라서 경고 처분이 적절하다고 판단됩니다.
3. **최종 판단:** 안전관리 기준의 불합리함은 심각한 문제이지만, 현재로서는 경미한 위반으로 볼 수 있으며, 재발 방지를 위한 교육과 경고가 필요하다고 판단하여 '경고' 처분을 추천합니다.


In [79]:
# 프롬프트에 처분 기준 조항 포함 시
answer = openai_chain.invoke('농업용 저수지 제방 안전등급 판정기준 불합리의 경우 어떤 처분을 내려야할까?')
print(answer)

## ⚖️ [새로운 감사 사례] 분석 및 처분 수준 추천

### 1. 유사 사례 분석 및 과거 처분 요약
농업용 저수지 제방 안전등급 판정기준의 불합리성에 대한 사례는 농어촌공사가 정밀안전진단을 수행하는 과정에서 발생한 문제로, 관련 법령에 따라 저수지의 안전성을 평가하는 기준이 불합리하다는 지적이 있었습니다. 이와 관련하여, 과거 유사 사례에서는 저수지의 안전성 평가 기준이 미흡하여 안전사고의 위험이 증가한 경우, 시정요구 및 개선요구가 내려진 바 있습니다.

### 2. 추천 처분 수준 및 근거
**[추천 처분 수준]**: 개선요구

**[법률적 근거]**
제25조 제5호에 해당하며, "감사 결과 법령상ㆍ제도상 또는 행정상 모순이 있거나 개선할 사항이 있다고 인정되는 경우"에 해당합니다.

**[추론 및 상세 근거]**
1. **유사성 인용:** 과거 사례에서 저수지 안전성 평가 기준의 불합리성으로 인해 시정요구 및 개선요구가 내려진 사례가 있으며, 이는 안전사고 예방을 위한 필수적인 조치로 간주되었습니다.
2. **심각성 비교:** 현재 사례는 저수지의 안전등급 판정기준이 불합리하다는 점에서 법령 및 제도상의 모순이 존재하며, 이는 향후 안전사고로 이어질 수 있는 중대한 문제입니다. 따라서, 단순한 경고나 주의요구보다는 개선요구가 적합합니다.
3. **최종 판단:** 법률적 기준과 유사 사례를 비교할 때, 현재 사례는 명백한 개선이 필요하며, 이를 통해 안전성을 확보할 수 있는 기회를 제공해야 하므로 개선요구가 가장 적합한 처분 수준으로 판단됩니다.


### [평가지표] (Evaluator, RAGAS 사용 가능)
1. **검색 성능 평가**: 사용자의 질문에 가장 관련성 높은 문서 청크를 얼마나 잘 찾아내는가?  
    - Recall: 정답 문서가 검색된 상위 K개의 결과 안에 포함되었는지 측정
    - Precision: 검색된 상위 K개의 결과 중 정답에 관련된 문서의 비율 측정
    - MAP: 검색된 결과의 순서를 고려하여 정밀도 평균화한 값 (높을수록 좋음)
    - MRR: 첫번째 정답 문서의 순위 역수를 평균화한 값 (1에 가까울 수록 좋음)
<br><br>
2. **생성 성능 평가**: LLM이 검색된 컨텍스트를 사용하여 얼마나 정확한 답변을 생성하는가?
    - Faithfulnsess: 답변이 컨텍스트만을 사용하여 작성되었는지, 환각 여부 확인 (높을수록 좋음)
    - Answer Relevancy: 답변이 사용자 질문에 얼마나 관련성이 높은지 (높을수록 좋음)
    - Context Utilization: 답변을 생성하는 데 검색된 컴텍스트가 얼마나 유용하게 쓰였는지 (높을수록 좋음)
<br><br>
3. **종합 성능 평가**: 최종 사용자가 느끼는 만족도와 시스템의 최종 목표 달성 여부 평가
    - LLM이 생성한 답변과 사람이 작성한 정답 간의 의미적 유사도 측정

##### Ragas의 한계
- LLM이 판단
    - 완전무결한 평가지표라고 할 수 없음
    - Judge model의 성능에 따라 점수 천차만별
    - LLM의 특성으로, 평가할 때마다 점수가 조금씩 달라지므로 측정의 신뢰도 저해 => 단순한 점수 차이가 실제 개선인지, 노이즈인지 판단 어려움
    - 객관성 부족
- **측정이 비용효율적 (CI/CD 파이프라인에 통합하여 품질 유지할 수 있게 해줌) => 이제 가장 큰 사용 이유**
- 사용 방법
    - 상대적 성능 비교로 활용
    - 그나마 신뢰도 높은 Judge Model 사용 (비용 고려 필요)
    - temperature를 0으로 설정(그럼에도 변동성 존재)
    - 대규모 데이터셋에 대해서 평가 (충분히 큰 수의 질문-답변 쌍에 대해 평가를 수행하여 개별 샘플의 측정 오류 희석)
    - 인간 검증 동반

##### Judge Model 영향
- gpt-4o-mini: 가끔 출력형태 불안정으로 오류 발생, 가장 저렴
- gpt-3.5-turbo: 안정적인 출력, gpt-4o-mini에 비해 3배 정도 비쌈
- gpt-4: 안정적인 출력, gpt-4o-mini에 비해 30배 정도 비쌈

In [121]:
# 질문 리스트
evaluation_questions = [
    '연구책임자가 직무관련자와 공모하여 수의계약을 체결한 대가로 1억 5천만원에 달하는금품을 수수한 경우 한국토지주택공사 사장에게 어떤 처분을 내려야해?', 
    '행정안전부에서 농업용 저수지 붕괴 등의 사고가 발생했음에도 사고 원인 조사를 실시하지 않은 경우 행정안전부 장관에게 어떤 처분을 내려야할까?',
    '보강공법 선정기준을 적용하지 않아 인버트 시공을 제외하여 피해가 발생하였으며,  터널 궤도 융기 발생에 대한 보수, 보강 대책이 부적정한 경우 국가철도공단 이사장에게 어떤 처분을 내려야해?',
    '이천시에서 입찰의 공정성을 저해하고, 특정 업체에 유리한 기술검토를 진행한 경우 이천시장에게 어떤 처분을 내려야해?',
    '지방자치단체에서 권한이 주어진 목적과 실체적 관련이 없는 다른 목적으로 인허가 협의 절차를 중단하여 인허가권을 남용한 경우 지방자체단체장에게 어떤 처분을 내려야할까?',
    '법령상 의무를 충분히 이행할 수 있는 상황이었음에도 경기도교육청 관내 각급 학교 중 35%에 해당하는 학교가 법령상 선임기한까지 기계설비유지관리자를 선임하지 않고 있던 경우, 지휘감독 업무를 소홀히 한경우 경기도 교육감에게 어떤 처분을 내려야해?',
    '보조금수령자가 지금목적과 다른 용도에 사용했음에도, 보조금을 환수하지 않은경우, 해당 지역 군수에게 어떤 처분을 내려야할까?',
    '철저한 조사 없이 퇴직자가 실질적 이사로 근무하는 업체와 단일 견적의 수의계약을 진행하고, 체결 후 결격 사유를 알게되었음에도 사후조치를 철저히 하지 않은 경우 도로공사 사장에게 어떤 추분을 내려야 할까?',
    '연구직공무원 및 공무직 채용과정에서 응시자격을 사후 변경하고, 정량평가를 잘못하는 등 서류전형 업무를 소홀히 한 경우, 해당 기관의 청장에게 어떤 처분을 내려야 할까?',
    '병무청에서 민원서비스 종합평가에서 최우수기관으로 선정될 목적으로 소속 직원들이 조직적으로 만족도 향상 정도 점수를 부풀려 우수기관 선정 및 대통령 표창과 포상금을 받은 경우 행정안전부장관에게 어떤 처분을 내려야해?'
]

In [122]:
# HyperClova Ragas 평가용
hc_chain_with_context = (
    # 1. 입력 문자열을 {'question': str}로 변환
    RunnableLambda(lambda x: {'question': x})
    |
    # 2. 검색 및 원본 컨텍스트 저장
    RunnablePassthrough.assign(
        docs=itemgetter("question") | hc_retriever 
    )
    |
    # 3. LLM이 사용할 포맷된 컨텍스트 생성
    RunnablePassthrough.assign(
        context=itemgetter("docs") | RunnableLambda(format_docs) 
    )
    |
    # 4. Prompt 입력 준비: 튜플이 아닌 딕셔너리 형태로 qa_prompt에 전달
    RunnablePassthrough.assign(
        llm_input={
            "question": itemgetter("question"),
            "context": itemgetter("context")
        }
    )
    |
    # 5. 답변 생성: llm_input 딕셔너리를 받아 Prompt에 전달
    RunnablePassthrough.assign(
        answer=itemgetter("llm_input") | qa_prompt | hc_llm | StrOutputParser()
    )
    |
    # 6. 최종 출력 정리: Ragas용 딕셔너리 반환
    {
        "question": itemgetter("question"),
        "answer": itemgetter("answer"),
        "raw_contexts_for_ragas": itemgetter("docs")
    }
)

In [123]:
faithfulness_li = []
answer_relevancy_li = []
context_precision_li = []

for i in range(10):
    results = []
    
    # 수정된 체인으로 호출
    for q in evaluation_questions:
        # result_dict는 딕셔너리
        result_dict = hc_chain_with_context.invoke(q) 
    
        results.append({
            'question': result_dict['question'],
            'answer': result_dict['answer'],
            'contexts': [doc.page_content for doc in result_dict['raw_contexts_for_ragas']],
            'reference': result_dict['answer']
        })
    
    df = pd.DataFrame(results)
    
    # Ragas 평가용 Dataset 객체 생성
    ragas_dataset = Dataset.from_pandas(df)
    
    judge_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    
    faithfulness.llm = judge_model
    answer_relevancy.llm = judge_model
    context_precision.llm = judge_model
    
    metrics = [
        faithfulness,       
        answer_relevancy, 
        context_precision
    ]
    
    result = evaluate(
        ragas_dataset,
        metrics=metrics,
    )

    faithfulness_li.append(np.mean(result['faithfulness']))
    answer_relevancy_li.append(np.mean(result['answer_relevancy']))
    context_precision_li.append(np.mean(result['context_precision']))

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Exception raised in Job[3]: ValidationError(1 validation error for StatementGeneratorOutput
  Invalid JSON: EOF while parsing a string at line 76 column 71 [type=json_invalid, input_value='{\n    "statements": [\n... 붕괴 사고의 원', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid)


...

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

In [71]:
print(f'faithfulness: {np.mean(result['faithfulness']):.4f}\nanswer_relevancy: {np.mean(result['answer_relevancy']):.4f}\ncontext_precision: {np.mean(result['context_precision']):.4f}')

faithfulness: 0.4731
answer_relevancy: 0.5183
context_precision: 0.8706


In [127]:
# OpenAI Ragas 평가용
openai_chain_with_context = (
    # 1. 입력 문자열을 {'question': str}로 변환
    RunnableLambda(lambda x: {'question': x})
    |
    # 2. 검색 및 원본 컨텍스트 저장
    RunnablePassthrough.assign(
        docs=itemgetter("question") | openai_retriever 
    )
    |
    # 3. LLM이 사용할 포맷된 컨텍스트 생성
    RunnablePassthrough.assign(
        context=itemgetter("docs") | RunnableLambda(format_docs) 
    )
    |
    # 4. Prompt 입력 준비: 튜플이 아닌 딕셔너리 형태로 qa_prompt에 전달
    RunnablePassthrough.assign(
        llm_input={
            "question": itemgetter("question"),
            "context": itemgetter("context")
        }
    )
    |
    # 5. 답변 생성: llm_input 딕셔너리를 받아 Prompt에 전달
    RunnablePassthrough.assign(
        answer=itemgetter("llm_input") | qa_prompt | openai_llm | StrOutputParser()
    )
    |
    # 6. 최종 출력 정리: Ragas용 딕셔너리 반환
    {
        "question": itemgetter("question"),
        "answer": itemgetter("answer"),
        "raw_contexts_for_ragas": itemgetter("docs")
    }
)

In [128]:
faithfulness_li = []
answer_relevancy_li = []
context_precision_li = []

for i in range(10):
    results = []
    
    ## 체인으로 호출
    for q in evaluation_questions:
        # result_dict는 딕셔너리
        result_dict = openai_chain_with_context.invoke(q) 
    
        results.append({
            'question': result_dict['question'],
            'answer': result_dict['answer'],
            'contexts': [doc.page_content for doc in result_dict['raw_contexts_for_ragas']],
            'reference': result_dict['answer']
        })
    
    df = pd.DataFrame(results)
    
    # Ragas 평가용 Dataset 객체 생성
    ragas_dataset = Dataset.from_pandas(df)
    
    judge_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    
    faithfulness.llm = judge_model
    answer_relevancy.llm = judge_model
    context_precision.llm = judge_model
    
    metrics = [
        faithfulness,       
        answer_relevancy,
        context_precision
    ]
    
    result = evaluate(
        ragas_dataset,
        metrics=metrics,
    )

    faithfulness_li.append(np.mean(result['faithfulness']))
    answer_relevancy_li.append(np.mean(result['answer_relevancy']))
    context_precision_li.append(np.mean(result['context_precision']))

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

In [76]:
print(f'faithfulness: {np.mean(result['faithfulness']):.4f}\nanswer_relevancy: {np.mean(result['answer_relevancy']):.4f}\ncontext_precision: {np.mean(result['context_precision']):.4f}')

faithfulness: 0.6005
answer_relevancy: 0.5225
context_precision: 0.8883


### Score 비교

- **Faithfulness(충실도): 환각 방지, 검색된 문서들 중 하나라도 답변을 뒷받침하는 문서가 있는가?**
    - 답변을 독립적인 문장 단위의 개별적인 명제로 나눔
    - 각 명제가 context(검색한 전체 문서)의 내용에 의해 논리적으로 지지 되는지 판단 (context 중 하나의 문서라도 지지된다면 1, 아니면 0)
    - 지지된 명제의 수를 전체 명제의 수로 나눔
<br><br>
- **Answer Relevancy(답변 적합성): LLM의 답변이 사용자의 원래 질문에 얼마나 집중했는가?**
    - 답변을 보고, 해당 답변을 유도했을 법한 가상의 질문 여러 개 생성 (일반적으로 3개)
    - 원래 질문과 가상 질문들 간의 의미적 유사도를 임베딩 모델을 통해 측정
    - 측정된 유사도 점수들의 평균 계산
<br><br>
- **Context Precision(맥락 정확도): 검색된 Context 안에 질문에 답하는 데 불필요한 노이즈 정보가 얼마나 섞여 있는가?**
    - 검색된 맥락을 개별 청크 또는 문장 단위로 나눔
    - 각 청크가 질문에 관련되어 답변에 필요한 정보인지 판단
    - 관련된 청크의 수를 전체 청크의 수로 나눔

### 프롬프트에 처벌 수준 미지정

#### **Judge Model: gpt-4o-mini**

- HyperClova
    - faithfulness: 0.5000
    - answer_relevancy: 0.7853

- OpenAI
    - faithfulness: 0.2718
    - answer_relevancy: 0.7988

#### **Judge Model: gpt-3.5-turbo**

- HyperClova
    - faithfulness: 0.5389
    - answer_relevancy: 0.7713

- OpenAI
    - faithfulness: 0.3667
    - answer_relevancy: 0.7924

### 프롬프트에 처벌 수준 지정

#### **Judge Model: gpt-4o-mini**

- HyperClova
    - faithfulness: 0.7305
    - answer_relevancy: 0.4156

- OpenAI
    - faithfulness: 0.1429
    - answer_relevancy: 0.7982

#### **Judge Model: gpt-3.5-turbo**

- HyperClova
    - faithfulness: 0.4231
    - answer_relevancy: 0.7697

- OpenAI
    - faithfulness: 0.4318
    - answer_relevancy: 0.3984

#### **Judge Model: gpt-4**

- HyperClova
    - faithfulness: 0.256
    - answer_relevancy: 0.792

- OpenAI
    - faithfulness: 0.1944
    - answer_relevancy: 0.7830

### 실행할 때마다 점수 달라지는 현상

- faithfulness: 0.5000
- answer_relevancy: 0.7853

- faithfulness: 0.6500
- answer_relevancy: 0.7791

- faithfulness: 0.4548
- answer_relevancy: 0.7936

- faithfulness: 0.2718
- answer_relevancy: 0.7988

### 프롬프트에 처벌 수준 미지정

#### HyperClova

In [105]:
# Faithfulness
print(f'Faithfulness 평균값: {np.nanmean(faithfulness_li):.4f}')
faithfulness_li

Faithfulness 평균값: 0.6254


[np.float64(0.590138888888889),
 np.float64(nan),
 np.float64(0.5548726763432645),
 np.float64(0.6078138528138528),
 np.float64(0.635892857142857),
 np.float64(nan),
 np.float64(0.7327106227106228),
 np.float64(nan),
 np.float64(0.7304334030262822),
 np.float64(0.5261363636363636)]

In [107]:
# answer_relevancy
print(f'answer_relevancy 평균값: {np.nanmean(answer_relevancy_li):.4f}')
answer_relevancy_li

answer_relevancy 평균값: 0.5166


[np.float64(0.5330434038730487),
 np.float64(0.4618568835599878),
 np.float64(0.4608603676519438),
 np.float64(0.5421817083842566),
 np.float64(0.5276706340531379),
 np.float64(0.603845880135794),
 np.float64(0.44876508751496036),
 np.float64(0.458510171179194),
 np.float64(0.5311873583421178),
 np.float64(0.5981672246316949)]

In [108]:
# context_precision
print(f'context_precision 평균값: {np.nanmean(context_precision_li):.4f}')
context_precision_li

context_precision 평균값: 0.9145


[np.float64(0.9366666666453611),
 np.float64(0.945555555534162),
 np.float64(0.950555555534037),
 np.float64(0.870555555534037),
 np.float64(0.870555555534037),
 np.float64(0.9455555555306618),
 np.float64(0.8599999999782917),
 np.float64(0.945555555526162),
 np.float64(0.94499999997925),
 np.float64(0.8745833333108717)]

#### OpenAI

In [112]:
# Faithfulness
print(f'Faithfulness 평균값: {np.nanmean(faithfulness_li):.4f}')
faithfulness_li

Faithfulness 평균값: 0.4664


[np.float64(0.5363528138528139),
 np.float64(0.51500777000777),
 np.float64(0.4348552754435107),
 np.float64(0.5486929736929737),
 np.float64(0.4662698412698412),
 np.float64(0.3625180375180375),
 np.float64(0.4082539682539682),
 np.float64(nan),
 np.float64(0.34868686868686866),
 np.float64(0.5770923520923522)]

In [113]:
# answer_relevancy
print(f'answer_relevancy 평균값: {np.nanmean(answer_relevancy_li):.4f}')
answer_relevancy_li

answer_relevancy 평균값: 0.6316


[np.float64(0.6852624078402454),
 np.float64(0.5442103817210435),
 np.float64(0.5326987241053301),
 np.float64(0.6678528863804186),
 np.float64(0.6883498511102121),
 np.float64(0.6293018427529509),
 np.float64(0.6926334052692568),
 np.float64(0.6774501306390072),
 np.float64(0.6839593617762662),
 np.float64(0.5138646383437312)]

In [114]:
# context_precision
print(f'context_precision 평균값: {np.nanmean(context_precision_li):.4f}')
context_precision_li

context_precision 평균값: 0.8864


[np.float64(0.8488888888678426),
 np.float64(0.9079166666457187),
 np.float64(0.8808333333130627),
 np.float64(0.9162499999801492),
 np.float64(0.9079166666457187),
 np.float64(0.8808333333130625),
 np.float64(0.8791666666467431),
 np.float64(0.8793055555359434),
 np.float64(0.9072222221941064),
 np.float64(0.8555555555352872)]

### 프롬프트에 처벌 수준 지정

#### HyperClova

In [124]:
# Faithfulness
print(f'Faithfulness 평균값: {np.nanmean(faithfulness_li):.4f}')
faithfulness_li

Faithfulness 평균값: 0.5523


[np.float64(0.5197604588394062),
 np.float64(0.5422167525457),
 np.float64(0.6522644009757013),
 np.float64(nan),
 np.float64(0.4714400584795322),
 np.float64(0.6118181818181817),
 np.float64(0.489084249084249),
 np.float64(0.4112501857410324),
 np.float64(0.7267365967365966),
 np.float64(0.5464048165793017)]

In [125]:
# answer_relevancy
print(f'answer_relevancy 평균값: {np.nanmean(answer_relevancy_li):.4f}')
answer_relevancy_li

answer_relevancy 평균값: 0.6644


[np.float64(0.4634267881070782),
 np.float64(0.6726889507343249),
 np.float64(0.7623098108787593),
 np.float64(0.6919665673918317),
 np.float64(0.6929547130823072),
 np.float64(0.6906811079453007),
 np.float64(0.6069352449403846),
 np.float64(0.6123995241314661),
 np.float64(0.7705778071757078),
 np.float64(0.6795993874044236)]

In [126]:
# context_precision
print(f'context_precision 평균값: {np.nanmean(context_precision_li):.4f}')
context_precision_li

context_precision 평균값: 0.9833


[np.float64(0.9566666666467499),
 np.float64(0.9866666666452361),
 np.float64(0.9916666666456111),
 np.float64(0.9887499999797813),
 np.float64(0.9916666666456111),
 np.float64(0.9887499999797813),
 np.float64(0.9866666666452361),
 np.float64(0.9866666666452361),
 np.float64(0.9693055555346517),
 np.float64(0.9866666666452361)]

#### OpenAI

In [129]:
# Faithfulness
print(f'Faithfulness 평균값: {np.nanmean(faithfulness_li):.4f}')
faithfulness_li

Faithfulness 평균값: 0.3880


[np.float64(0.3221428571428572),
 np.float64(0.3801098901098901),
 np.float64(0.32212842712842715),
 np.float64(0.4081938821412505),
 np.float64(0.3590277777777778),
 np.float64(0.33679065086959825),
 np.float64(0.39888680033416873),
 np.float64(0.49954790823211875),
 np.float64(0.42806637806637815),
 np.float64(0.4246794871794871)]

In [130]:
# answer_relevancy
print(f'answer_relevancy 평균값: {np.nanmean(answer_relevancy_li):.4f}')
answer_relevancy_li

answer_relevancy 평균값: 0.5250


[np.float64(0.5956565403333716),
 np.float64(0.6045480582320883),
 np.float64(0.45455850586344687),
 np.float64(0.6657603705696366),
 np.float64(0.45505924312744633),
 np.float64(0.6012864929001872),
 np.float64(0.45072349084581625),
 np.float64(0.3039234434195962),
 np.float64(0.6655801708208977),
 np.float64(0.45334297828169595)]

In [131]:
# context_precision
print(f'context_precision 평균값: {np.nanmean(context_precision_li):.4f}')
context_precision_li

context_precision 평균값: 0.8841


[np.float64(0.8705555555342871),
 np.float64(0.8974999999779651),
 np.float64(0.9138888888682593),
 np.float64(0.8655555555348009),
 np.float64(0.9155555555351341),
 np.float64(0.9045833333122465),
 np.float64(0.835416666646948),
 np.float64(0.9037499999809062),
 np.float64(0.8612499999792742),
 np.float64(0.8724999999792153)]